# Create a SDS file structure by retrieving data from a FDSN web server

If you need a SDS file structure, but only have a FDSN web server to provide data, then you can run a series of data requests to the server and create SDS.

For GeoNet, we have the particular issue of two FDSN web servers. You can get around this by retrieving from each server and merging the streams.

In [29]:
from obspy.core import UTCDateTime, Stream
from obspy.clients.fdsn import Client
import pandas as pd
import os

In [30]:
#GeoNet's FDSN web servers
arc_client = 'http://service.geonet.org.nz'
nrt_client = 'http://service-nrt.geonet.org.nz'

In [31]:
#SDS file structure to create
sds = '/home/sherburn/SDS/test_sds'

In [32]:
#GeoNet station list
# stations = ['KUTZ', 'WPRZ', 'ALRZ', 'ARAZ', 'HRRZ', 'PRRZ', 'THQ2']
stations = ['THQ2']
net = 'NZ'
loc = '10'
components = ['EHZ', 'EHN', 'EHE', 'EH1', 'EH2']

In [33]:
#get data for one day, try both servers
def daywave(date, net, sta, loc, cmp):
    global st
    
    st = Stream()
    starc = Stream()
    stnrt = Stream()
    
    time = UTCDateTime(date)
    
    client = Client(arc_client)
    try:
        starc = client.get_waveforms(net, sta, loc, cmp, time, time + 60)
    except:
        pass

    client = Client(nrt_client)
    try:
        stnrt = client.get_waveforms(net, sta, loc, cmp, time, time + 60)
    except:
        pass
    
    st = starc + stnrt
    st.merge(fill_value = 'interpolate')

In [35]:
#date range for waveform data, dates are inclusive
#edit next two lines to set date range
date1 = '20190501'
date2 = '20190512'

In [36]:
for sta in stations:
    for cmp in components:
        for date in pd.date_range(start=date1, end=date2, freq='D'):
#             print (sta, cmp, loc, net, date)
            
            daywave(date, net, sta, loc, cmp)
            
            if st.count() > 0:
                year = date.timetuple().tm_year
                yday = date.timetuple().tm_yday
                folder = os.path.join(sds, str(year), net, sta, cmp+'.D')
                if not os.path.exists(folder):
                    os.makedirs(folder)
                file = os.path.join(folder, net+'.'+sta+'.'+loc+'.'+cmp+'.D.'+str(year)+'.'+str(yday).zfill(3))
                print ('writing:', file)
                st.write(file, format="MSEED")          

writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.121
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.122
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.123
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.124
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.125
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.126
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.127
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.128
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.129
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.130
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.131
writing: /home/sherburn/SDS/test_sds/2019/NZ/THQ2/EHZ.D/NZ.THQ2.10.EHZ.D.2019.132
writing: /home/s